# Querying Metadata with GraphQL

In this notebook we explore searching for metadata from the GraphQL API. The GraphQL API provides a method to programmatically extract a JSON representation of the meta data from the API. It has the advantage over REST that it can prevent under and over fetching data.

First we load some python dependancies that we will use as part of this notebook and set the variable `ENDPOINT_URL` to the location of the GraphQL API.

In [12]:
import yaml
import pandas as pd

with open('config.yml') as file_handle:
    config = yaml.load(file_handle, yaml.FullLoader)

ENDPOINT_URL = config['graphql_api']

Setup a `gql` client. This is used to query the graphql API

In [8]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Select your transport with a defined url endpoint
transport = RequestsHTTPTransport(url=ENDPOINT_URL)
# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

## Querying Shots with GraphQL

With GraphQL you can query exactly what you want, rather than having to recieve the whole table from the database This is useful in cases where the whole table has many columns, but you are interested in just a subset of them.

The GraphQL endpoint is located at `/graphql`. You can find the documentation and an interactive query explorer at the URL below:

In [9]:
print(f"GraphQL API endpoint: {ENDPOINT_URL}")

GraphQL API endpoint: http://209.97.185.27:8081/graphql


Unlike the REST API which uses HTTP `GET` requests to return data, with GraphQL we use HTTP `POST` to post our query to the API.

Here is a simple example of getting some shot data from the GraphQL API. We need to explicity state what information we want to return from the API. Here we are asking for:
 - the shot ID
 - the timestamp that the shot was taken
 - the preshot description
 - the divertor configuration

In [15]:
# Write our GraphQL query.
query = gql("""
query {
    all_shots  {
        shots {
            shot_id
            timestamp
            preshot_description
            divertor_config
        }
    }
}
""")

# # Query the API and get a JSON response
result = client.execute(query)
shots = result['all_shots']['shots']
df = pd.DataFrame(shots)
df.head()

,shot_id,timestamp,preshot_description,divertor_config
0,11695,2004-12-13T11:54:00,\n0.1T TF SHOT\n,conventional
1,11696,2004-12-13T12:07:00,\nSTANDARD 0.3T TF SHOT\n,conventional
2,11697,2004-12-13T12:19:00,\nRAISE TO 0.5T\n,conventional
3,11698,2004-12-13T12:31:00,\nRAISE TO .56T\n,conventional
4,11699,2004-12-13T12:45:00,\nRAISE TO .58T\n,conventional


### Searching & Filtering Data
We can also supply query parameters to GraphQL, such as limiting the number of returned values or filtering by value. Here we are limiting the first 3 values and we are selcting only shots from the M9 campaign.

In [16]:
# Write our GraphQL query.
query = gql("""
query ($campaign: String!) {
    all_shots (limit: 3, where: {campaign: {eq: $campaign}}) {
        shots {
            shot_id
            timestamp
            preshot_description
            divertor_config
        }
    }
}
""")
# Query the API and get a JSON response
result = client.execute(query, variable_values={"campaign": "M9"})
df = pd.DataFrame(result['all_shots']['shots'])
df.head()

,shot_id,timestamp,preshot_description,divertor_config
0,28390,2012-03-06T14:47:00,"\nBC5, 300 ms, 3 V. D2 plenum 1536 mbar. For r...",conventional
1,28391,2012-03-06T14:52:00,"\nBC5, 300 ms, 5 V. D2 plenum 1536 mbar. For r...",conventional
2,28392,2012-03-06T15:03:00,"\nHL11, 300 ms, 2 V. He plenum 1047.\n",conventional


### Nested Queries

One feature which makes GraphQL much more powerful that REST is that you may perform nested queries to gather different subsets of the data. For example, here we are going to query for all datasets with "AMC" in the name, and query for information about shots associated with this dataset.

In [17]:
# Write our GraphQL query.
query = gql("""
query ($signal: String!) {
    all_signal_datasets (limit: 3, where: {name: {contains: $signal}}) {
        signal_datasets {
          name
          url
          shots (limit:3) {
            shot_id
            timestamp
            divertor_config
          }
        }
    }
}
""")
# Query the API and get a JSON response
client.execute(query, variable_values={"signal": "AMC"})

{'all_signal_datasets': {'signal_datasets': [{'name': 'AMC_P3U_COIL_CURRENT',
    'url': 's3://mast/AMC_P3U_COIL_CURRENT.zarr',
    'shots': [{'shot_id': 28412,
      'timestamp': '2013-03-06T15:13:00',
      'divertor_config': 'conventional'},
     {'shot_id': 28415,
      'timestamp': '2013-03-07T15:11:00',
      'divertor_config': 'conventional'},
     {'shot_id': 28416,
      'timestamp': '2013-03-07T15:27:00',
      'divertor_config': 'conventional'}]},
   {'name': 'AMC_P2OL_FEED_CURRENT',
    'url': 's3://mast/AMC_P2OL_FEED_CURRENT.zarr',
    'shots': [{'shot_id': 23819,
      'timestamp': '2009-12-09T10:04:00',
      'divertor_config': 'conventional'},
     {'shot_id': 23820,
      'timestamp': '2009-12-09T10:24:00',
      'divertor_config': 'conventional'},
     {'shot_id': 23821,
      'timestamp': '2009-12-09T10:48:00',
      'divertor_config': 'conventional'}]},
   {'name': 'AMC_ERROR_FIELD_05',
    'url': 's3://mast/AMC_ERROR_FIELD_05.zarr',
    'shots': [{'shot_id': 29516,

### Pagination in GraphQL

GraphQL queries are paginated. You may access other entries by including the page metadata and it's associated elements. Here's an example of getting paginated entries:

In [19]:
def do_query(cursor: str = None):
    query = gql("""
    query ($cursor: String) {
        all_shots (limit: 3, where: {campaign: {contains: "M9"}}, cursor: $cursor) {
            shots {
                shot_id
                timestamp
                preshot_description
                divertor_config
            }
            page_meta {
              next_cursor
              total_items
              total_pages
            }
        }
    }
    """)
    return client.execute(query, {'cursor': cursor})


def iterate_responses():
    cursor = None
    while True:
        response = do_query(cursor)
        yield response
        cursor = response['all_shots']['page_meta']['next_cursor']
        if cursor is None:
            return

responses = iterate_responses()
print(next(responses))
print(next(responses))
print(next(responses))

{'all_shots': {'shots': [{'shot_id': 28390, 'timestamp': '2012-03-06T14:47:00', 'preshot_description': '\nBC5, 300 ms, 3 V. D2 plenum 1536 mbar. For reference.\n', 'divertor_config': 'conventional'}, {'shot_id': 28391, 'timestamp': '2012-03-06T14:52:00', 'preshot_description': '\nBC5, 300 ms, 5 V. D2 plenum 1536 mbar. For reference.\n', 'divertor_config': 'conventional'}, {'shot_id': 28392, 'timestamp': '2012-03-06T15:03:00', 'preshot_description': '\nHL11, 300 ms, 2 V. He plenum 1047.\n', 'divertor_config': 'conventional'}], 'page_meta': {'next_cursor': 'Mw==', 'total_items': 2082, 'total_pages': 694}}}
{'all_shots': {'shots': [{'shot_id': 28393, 'timestamp': '2012-03-06T15:09:00', 'preshot_description': '\nHL11, 300 ms, 3 V. He plenum 1047.\n', 'divertor_config': 'conventional'}, {'shot_id': 28394, 'timestamp': '2012-03-06T15:13:00', 'preshot_description': '\nHL11, 300 ms, 5 V. He plenum 1047.\n', 'divertor_config': 'conventional'}, {'shot_id': 28395, 'timestamp': '2012-03-06T15:21:0